In [20]:
# Convolutional Neural Network(Classification)
# 卷积神经网络的使用
# 数据集：https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy    food-11.zip

In [21]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm import tqdm

In [22]:
## Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.
## torchvision提供了很多图像处理，数据包装和数据增强工具

## Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.
## 由于数据通过label存在不同的文件夹中，可以直接使用DatasetFolder进行数据包装

## Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [23]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [24]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [25]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            # in_channel必须是3 因为输入图像是RGB三个channel
            # out_channel自定义
            # kernel_size 3 * 3
            # stride 步长为1
            # padding 设为1 在四周每个边都补 padding mode默认是zero 所以是补0
            nn.BatchNorm2d(64),
            # batchNorm是将数据维度归一化的方法 从而产生好的梯度值 batchNorm一般放在conv之后active之前
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            # MaxPool 将数据池化降维

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            # conv2d让输入的height和width没有改变
            # MaxPool2d的参数2，使得height和width每次缩小一倍
            # MaxPool2d的参数4，使得height和width每次缩小两倍
            # 所以在经历三个MaxPool2d之后，128的height和width变为了8*8
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)
        # flatten的dim=1即保留第一维(batch) 让后续拉成一维

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

In [26]:
# ------------------------------------------------------------------------------------------
# 注意该部分为半监督学习过程中将无label的数据创建为有label的数据加以训练的过程

def get_pseudo_labels(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 构建Dataloader
    dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=False)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    # ------------------------------------------------------------------------------ #
    tr = torch.tensor([])
    tl = torch.tensor([])
    # ------------------------------------------------------------------------------- #
    # Iterate over the dataset by batches.
    for batch in tqdm(dataloader):
        img, _ = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)

        # -------------------------------------------------------------------------------------#
        probs_argmax = probs.argmax(dim=-1).cpu()
        probs_max = probs.max(dim=-1).values
        probs_max = probs_max.cpu().numpy().tolist()
        feat = []
        for idx,p in enumerate(probs_max):
          if p > threshold:
            feat.append(idx)
                # 将概率值大于threshold的数据的index拿出来 作为扩展数据集
        #new_batch = [img[feat],probs_argmax[feat]]
        #print("new_batch:",new_batch)
        tr = torch.cat((tr,img[feat]))
        tl = torch.cat((tl,feat)) # !!! 加的应该是feat不是probs_argmax[feat]吧 !!!
    new_data = ConcatDataset([tr,tl])
    print("new_data length:",len(new_data))
    # ------------------------------------------------------------------------------------------#
        # ---------- TODO ----------
        # Filter the data and construct a new dataset.

    # # Turn off the eval mode.
    model.train()
    return dataset

In [27]:
# os.environ['WANDB_CONSOLE'] = 'off'

# "cuda" only when GPUs are available.
device = "cuda:3" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 10

# Whether to do semi-supervised learning.
do_semi = False

for epoch in range(n_epochs):
    # print('epoch:',epoch)
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if do_semi:
        # 如果使用了半监督学习 每次迭代时使用上次模型计算non-label数据的label 也作为训练
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(unlabeled_set, model)

        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=64, pin_memory=True)

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    batch_num = 0
    # Iterate the training set by batches.
    for batch in tqdm(train_loader):
        batch_num = batch_num + 1
        # print('batch_num:',batch_num)

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))
        # 无需在计算交叉熵之前对结果再加一层softmax 因为交叉熵loss内部会进行此计算
        # 交叉熵的第一个参数是model的输出维度是(batch, classes)
        # (批次，每个类别的计算值 是(3.1,5.3,7.8...) 没有用sigmoid转化为概率 0.1,0.1,0.69...) 这个操作交叉熵内部会做
        # 交叉熵的第二个参数是label 维度是(batch,classes) (批次， 每个类别的数值 1,2,3,..)

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        # clip_grad_norm 梯度裁剪 让每一次训练的结果都不过分的依赖某一部分神经元，
        # 在训练的时候随机忽略一些神经元和神经的链接，使得神经网络变得不完整， 是解决过拟合的一种方法。
        # 注意这个方法只在训练的时候使用，在测试的时候验证和测试的时候不用。

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

  0%|          | 0/25 [00:00<?, ?it/s]

epoch: 0


 12%|█▏        | 3/25 [00:01<00:20,  1.05it/s]

batch_num: 1
batch_num: 2
batch_num: 3
batch_num: 4


 32%|███▏      | 8/25 [00:02<00:09,  1.80it/s]

...

batch_num: 23
batch_num: 24
batch_num: 25


  0%|          | 0/6 [00:00<?, ?it/s]

[ Train | 010/010 ] loss = 0.90847, acc = 0.70781


100%|██████████| 6/6 [00:02<00:00,  2.18it/s]

[ Valid | 010/010 ] loss = 1.64917, acc = 0.48698


In [28]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())


100%|██████████| 27/27 [00:24<00:00,  1.10it/s]
